In [21]:
import pandas as pd



In [22]:
'''
quota = pd.read_html('quota.html', match= "Quota")
print(len(quota))
df = quota[0]
df.head()
df.columns = ['Department', 'Status', "Quota", "Current","a"]
df.head()
df.drop("a",axis=1, inplace=True)
df
df.drop(labels=range(0,2), inplace=True)
df = df.reset_index(drop=True)
df
'''

'''
import requests
from bs4 import BeautifulSoup
 
 
url = 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
for link in soup.find_all('a'):
    print(link.get('href'))
'''

1


In [28]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

driver1 = webdriver.Chrome()
driver1.get("https://registration.boun.edu.tr/buis/General/schedule.aspx?p=semester")

wait = WebDriverWait(driver1, 10)
element1 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="submit"]')))
element_id1 = element1.get_attribute('id')
go_button = wait.until(EC.presence_of_element_located((By.ID, element_id1)))


go_button.click()

driver1.implicitly_wait(3)

link_elements = driver1.find_elements(By.TAG_NAME, 'a')

urls = []

for link in link_elements:
    urls.append(link.get_attribute('href'))

driver1.quit()


['https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=DSAI&bolum=ARTIFICIAL+INTELLIGENCE', 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=ASIA&bolum=ASIAN+STUDIES', 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=ASIA&bolum=ASIAN+STUDIES+WITH+THESIS', 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=ATA&bolum=ATATURK+INSTITUTE+FOR+MODERN+TURKISH+HISTORY', 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=AUTO&bolum=AUTOMOTIVE+ENGINEERING', 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=BM&bolum=BIOMEDICAL+ENGINEERING', 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=BIS&bolum=BUSINESS+INFORMATION+SYSTEMS', 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=BIS&bolum=BUSINESS+INFORMATION+SYSTEMS+(WITH+THESIS)', 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023

In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

dfs = []

for url in urls:
    driver.get(url)
    info_links = driver.find_elements(By.LINK_TEXT,"Info")
    for info_link in info_links:
        info_link.click()
        # Switch to the new window
        new_window = driver.window_handles[-1]
        driver.switch_to.window(new_window)
        try:
            # Find the table element
            table = driver.find_element(By.TAG_NAME, "table")
            # Convert the table content to a DataFrame
            df2 = pd.read_html(table.get_attribute('outerHTML'))[0]
            strongs = driver.find_elements(By.TAG_NAME, "strong")
            strongs_text = [strong.text for strong in strongs]
            strongs_text_repeated = np.repeat(strongs_text, len(df2))
            df2["Code"] = strongs_text_repeated[:len(df2)]
            dfs.append(df2)
        except IndexError:
            # No table found, move on to the next link
            pass
        finally:
            # Close the new window
            driver.close()
            # Switch back to the main window
            main_window = driver.window_handles[0]
            driver.switch_to.window(main_window)


result = pd.concat(dfs, ignore_index=True)

In [31]:
result = result[["Code",0,1,2,3,4]]

,Code,0,1,2,3,4
0,DSAI301.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,DSAI301.01,Department,Statu,Quota,Current,NaN
2,DSAI301.01,ALL,ALL,Consent Of Instructor,0,NaN
3,ASIA502.02,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
4,ASIA502.02,Department,Statu,Quota,Current,NaN
...,...,...,...,...,...,...
10569,LIT 311.01,ALL,ALL,Consent Of Instructor,0,NaN
10570,LIT 311.01,ALL,UNDERGRADUATE,40,65,NaN
10571,LIT 412.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
10572,LIT 412.01,Department,Statu,Quota,Current,NaN


In [32]:
result.drop(result.columns[[5]], axis=1, inplace=True)

C:\Users\Aykut\AppData\Local\Temp\ipykernel_6792\1594288655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(result.columns[[5]], axis=1, inplace=True)


,Code,0,1,2,3
0,DSAI301.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,DSAI301.01,Department,Statu,Quota,Current
2,DSAI301.01,ALL,ALL,Consent Of Instructor,0
3,ASIA502.02,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
4,ASIA502.02,Department,Statu,Quota,Current
...,...,...,...,...,...
10569,LIT 311.01,ALL,ALL,Consent Of Instructor,0
10570,LIT 311.01,ALL,UNDERGRADUATE,40,65
10571,LIT 412.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
10572,LIT 412.01,Department,Statu,Quota,Current


In [33]:
result.columns = ['Code', 'Department', 'Status', 'Quota', 'Current Quota']

,Code,Department,Status,Quota,Current Quota
0,DSAI301.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,DSAI301.01,Department,Statu,Quota,Current
2,DSAI301.01,ALL,ALL,Consent Of Instructor,0
3,ASIA502.02,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
4,ASIA502.02,Department,Statu,Quota,Current
...,...,...,...,...,...
10569,LIT 311.01,ALL,ALL,Consent Of Instructor,0
10570,LIT 311.01,ALL,UNDERGRADUATE,40,65
10571,LIT 412.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
10572,LIT 412.01,Department,Statu,Quota,Current


In [34]:
words = ["Departmental Quotas:", "Department", "OZELLISANS", "PHD","MASTER","OZELMASTER","OZELPHD"]

result = result[result.Department.isin(words) == False]
result = result[result.Status.isin(words) == False]
result.reset_index(drop=True, inplace=True)
result

,Code,Department,Status,Quota,Current Quota
0,DSAI301.01,ALL,ALL,Consent Of Instructor,0
1,ASIA502.02,ALL,ALL,Consent Of Instructor,2
2,ASIA519.02,ALL,ALL,Consent Of Instructor,5
3,ASIA521.02,ALL,ALL,Consent Of Instructor,1
4,ASIA527.02,ALL,ALL,Consent Of Instructor,2
...,...,...,...,...,...
3053,FA 49F.01,ALL,ALL,Consent Of Instructor,39
3054,FA 49S.01,ALL,ALL,Consent Of Instructor,16
3055,LIT 311.01,ALL,ALL,Consent Of Instructor,0
3056,LIT 311.01,ALL,UNDERGRADUATE,40,65


In [35]:
'''
import openpyxl

file_name = "AllTest.xlsx"

result.to_excel(file_name)
'''